In [1]:
from nba_api.stats.static import teams, players

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow.keras as tfk
from scikeras.wrappers import KerasRegressor

#import scikeras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, Masking, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

In [2]:
gameLogs = pd.read_csv('gameLogs.csv').drop(columns="Unnamed: 0")

home_teams = gameLogs[gameLogs['HOME_FLAG'] == 1]
away_teams = gameLogs[gameLogs['HOME_FLAG'] == 0]

# Unire i DataFrame per ottenere le informazioni richieste
history_df = pd.DataFrame({
    'Home_Team': home_teams['NICKNAME'].values,
    'Away_Team': away_teams['NICKNAME'].values,
    'Season': home_teams['SEASON'].values,
    'Game_Date': home_teams['GAME_DATE'].values,
    'Game_ID': home_teams['GAME_ID'].values,
    'y': (home_teams['W'].values).astype(int),
})

#history_df

In [4]:
from nba_api.stats.endpoints import teamdetails, teamplayerdashboard, playercareerstats
import json

selected_columns = ['GP', 'GS', 'MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
num_stats = len(selected_columns)


#for i, stat in enumerate(selected_columns):
    #df[f"Home_Stat_{stat}"] = 0.0
    #df[f"Away_Stat_{stat}"] = 0.0


for i, row in history_df.iterrows():
    home_team = row['Home_Team']
    away_team = row['Away_Team']
    teams_query= [home_team, away_team]
    season = row['Season']
    
    array_1 = np.array([0.0] * num_stats)
    array_2 = np.array([0.0] * num_stats)
    teams_stats = [array_1, array_2]
    
    for idx, team in enumerate(teams_query):
        t1 = teams.find_teams_by_full_name(team)
        team_id = t1[0]['id']
        print(team_id)

        t = teamplayerdashboard.TeamPlayerDashboard(team_id=team_id, season=season)
        data_player = json.loads(t.get_json())['resultSets']

        # Extract the 'rowSet' for 'PlayersSeasonTotals'
        players_data = next(item for item in data_player if item['name'] == 'PlayersSeasonTotals')['rowSet']
        # Extract 'PLAYER_ID' for each player
        player_ids = [player[1] for player in players_data]
        print(f"{player_ids}\n")

        for player_id in player_ids:
            career = playercareerstats.PlayerCareerStats(player_id=player_id)
            df_single_player_carrer = career.get_data_frames()[0]
            #last_row_stats = df_single_player_carrer.iloc[-1, df_single_player_carrer.columns.get_loc('PLAYER_AGE') + 1:].tolist()
            last_row_stats = df_single_player_carrer[selected_columns].iloc[-1].tolist()
            print(f"{last_row_stats}")
            teams_stats[idx] += np.array(last_row_stats)
    
    # Aggiungere le statistiche al DataFrame
    for j, stat in enumerate(selected_columns):
        history_df.loc[i, f'Home_Stat_{stat}'] = teams_stats[0][j]
        history_df.loc[i, f'Away_Stat_{stat}'] = teams_stats[1][j]
        
    print(f"Finito row {i}")

1610612737
[203992, 1629164, 1628981, 1629629, 203991, 201568, 1629631, 1628381, 1628989, 1627739, 101150, 1630233, 1630168, 200765, 1630219, 203524, 203503, 1629027]

[79.0, 33.0, 2401.0, 0.428, 0.374, 0.921, 53.0, 219.0, 272.0, 247.0, 96.0, 26.0, 112.0, 179.0, 1333.0]
[36.0, 5.0, 502.0, 0.416, 0.345, 0.632, 14.0, 53.0, 67.0, 91.0, 25.0, 1.0, 36.0, 39.0, 171.0]
[45.0, 2.0, 682.0, 0.583, 0.0, 0.667, 59.0, 134.0, 193.0, 46.0, 25.0, 28.0, 44.0, 107.0, 282.0]
[48.0, 26.0, 984.0, 0.389, 0.336, 0.759, 27.0, 72.0, 99.0, 48.0, 47.0, 14.0, 31.0, 64.0, 260.0]
[73.0, 73.0, 1883.0, 0.571, 0.0, 0.631, 335.0, 441.0, 776.0, 91.0, 43.0, 106.0, 74.0, 164.0, 836.0]


KeyboardInterrupt: 

In [ ]:
#all_teams_df.to_csv("all_players_db.csv", index=False)